## Imports

In [1]:
import pandas as pd
from pyprojroot.here import here
import itertools
from fuzzywuzzy import fuzz, process

C:\Users\35196\PycharmProjects\pythonProject1\.venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## 

In [2]:
zonas = pd.read_csv(here('data/zonasConcelhos.csv'))
zonas

,lat,long,zona,concelho
0,37.0122,-7.93411,A,Faro
1,37.0133,-7.92329,A,Faro
2,37.0138,-7.92776,A,Faro
3,37.0140,-7.91955,A,Faro
4,37.0150,-7.93245,A,Faro
...,...,...,...,...
10501,42.1062,-8.21306,C,Melgaço
10502,42.1108,-8.27178,C,Melgaço
10503,42.1153,-8.25803,C,Melgaço
10504,42.1291,-8.24435,C,Melgaço


In [3]:
###### Para verificar se está certo ######
#count distinct concelhos e zonas agrupados
zonas.groupby(["zona", "concelho"]).ngroups # está certo mais para a frente no ficheiro

443

#### Tirar ilhas e ver todos os concelhos existentes

In [4]:
all_concelhos = pd.read_excel(here('data/concelhos-metadata.xlsx'))
all_concelhos = all_concelhos.query("dicofre <= 2000") # retirar as ilhas
all_concelhos = all_concelhos[["designacao"]].rename(columns={"designacao": "concelho"})
all_zonas = ["A", "B", "C"]
all_concelhos

,concelho
0,Águeda
1,Albergaria-a-Velha
2,Anadia
3,Arouca
4,Aveiro
...,...
273,Tarouca
274,Tondela
275,Vila Nova de Paiva
276,Viseu


#### Todas as combinações possíveis de zonas e concelhos (para contar os complete cases, mesmo não existindo)

In [5]:
complete = pd.DataFrame(list(itertools.product(all_zonas, all_concelhos["concelho"])), columns=["zona", "concelho"])
complete.sort_values(by=['zona', 'concelho'])
complete

,zona,concelho
0,A,Águeda
1,A,Albergaria-a-Velha
2,A,Anadia
3,A,Arouca
4,A,Aveiro
...,...,...
829,C,Tarouca
830,C,Tondela
831,C,Vila Nova de Paiva
832,C,Viseu


### Fazer count distinct dos concelhos com as zonas agrupadas

In [6]:
zonas_concelhos_group = zonas.groupby(["zona", "concelho"]).size().to_frame('count').reset_index()
zonas_concelhos_group

,zona,concelho,count
0,A,Almada,102
1,A,Amadora,94
2,A,Aveiro,112
3,A,Barreiro,41
4,A,Beja,48
...,...,...,...
438,C,Vouzela,16
439,C,Águeda,20
440,C,Évora,4
441,C,Ílhavo,11


In [7]:
complete_zonas = pd.merge(complete, zonas_concelhos_group, on=["zona", "concelho"], how="left")
complete_zonas["count"] = complete_zonas["count"].fillna(0)
complete_zonas.sort_values(by=['concelho', 'zona'])

,zona,concelho,count
196,A,Abrantes,0.0
474,B,Abrantes,26.0
752,C,Abrantes,20.0
117,A,Aguiar da Beira,0.0
395,B,Aguiar da Beira,0.0
...,...,...,...
287,B,Ílhavo,34.0
565,C,Ílhavo,11.0
142,A,Óbidos,0.0
420,B,Óbidos,0.0


In [8]:
print(len(zonas.query("concelho == 'Lisboa' and zona == 'A'")))# deu certo
print(len(zonas.query("concelho == 'Lisboa' and zona == 'B'"))) # deu certo
# para testar em outros casos substituir pelos valores dos campos :)

# len(zonas.query("concelho == '...' and zona == '...'"))

880
0


### Fazer pivot table para ser mais fácil no futuro

In [9]:
zonas_final = complete_zonas.pivot_table(values='count', index = ['concelho'] ,columns = ['zona'], aggfunc='sum', fill_value=0).reset_index()
zonas_final.columns= ["concelho", "count_zonaA", "count_zonaB", "count_zonaC"]
zonas_final

,concelho,count_zonaA,count_zonaB,count_zonaC
0,Abrantes,0.0,26.0,20.0
1,Aguiar da Beira,0.0,0.0,5.0
2,Alandroal,0.0,0.0,4.0
3,Albergaria-a-Velha,0.0,12.0,7.0
4,Albufeira,0.0,47.0,23.0
...,...,...,...,...
273,Vouzela,0.0,0.0,16.0
274,Águeda,0.0,7.0,20.0
275,Évora,70.0,18.0,4.0
276,Ílhavo,0.0,34.0,11.0


### Fazer count distinct dos concelhos apenas

In [10]:
distinct_concelhos = zonas.groupby(["concelho"]).size().to_frame('total_concelho').reset_index()
distinct_concelhos

,concelho,total_concelho
0,Abrantes,46
1,Aguiar da Beira,5
2,Alandroal,4
3,Albergaria-a-Velha,19
4,Albufeira,70
...,...,...
268,Vouzela,16
269,Águeda,27
270,Évora,92
271,Ílhavo,45


## Fazer percentagens

In [11]:
zonas_com_percentagens = pd.merge(zonas_final, distinct_concelhos, on="concelho", how="left")
# colocar 0 nos valores nulos, depois do merge porque distinct_concelhos não tem todos os concelhos
zonas_com_percentagens["total_concelho"] = zonas_com_percentagens["total_concelho"].fillna(0)

zonas_com_percentagens["percentagem_zonaA"] = zonas_com_percentagens["count_zonaA"] / zonas_com_percentagens["total_concelho"]
zonas_com_percentagens["percentagem_zonaB"] = zonas_com_percentagens["count_zonaB"] / zonas_com_percentagens["total_concelho"]
zonas_com_percentagens["percentagem_zonaC"] = zonas_com_percentagens["count_zonaC"] / zonas_com_percentagens["total_concelho"]


zonas_com_percentagens[["percentagem_zonaA", "percentagem_zonaB", "percentagem_zonaC"]] = zonas_com_percentagens[["percentagem_zonaA", "percentagem_zonaB", "percentagem_zonaC"]].astype(float)
zonas_com_percentagens[["count_zonaA", "count_zonaB", "count_zonaC", "total_concelho"]] = zonas_com_percentagens[["count_zonaA", "count_zonaB", "count_zonaC", "total_concelho"]].astype(int)

In [12]:
zonas_com_percentagens.sort_values(by="total_concelho", ascending=True)

,concelho,count_zonaA,count_zonaB,count_zonaC,total_concelho,percentagem_zonaA,percentagem_zonaB,percentagem_zonaC
231,Tavira,0,0,0,0,NaN,NaN,NaN
75,Constância,0,0,0,0,NaN,NaN,NaN
40,Barrancos,0,0,0,0,NaN,NaN,NaN
186,Ponte de Sor,0,0,0,0,NaN,NaN,NaN
20,Alter do Chão,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
220,Sintra,97,60,43,200,0.485000,0.300000,0.215000
49,Braga,240,2,106,348,0.689655,0.005747,0.304598
257,Vila Nova de Gaia,118,247,3,368,0.320652,0.671196,0.008152
190,Porto,494,0,0,494,1.000000,0.000000,0.000000


In [13]:
percentage_missing_mask = zonas_com_percentagens[['percentagem_zonaA', 'percentagem_zonaB', 'percentagem_zonaC']].isnull().any(axis=1)
# percentage_missing_mask
zonas_com_percentagens.loc[percentage_missing_mask, 'percentagem_zonaC'] = 1
zonas_com_percentagens.loc[percentage_missing_mask, 'count_zonaC'] = 1
zonas_com_percentagens.loc[percentage_missing_mask, 'total_concelho'] = 1
zonas_com_percentagens.loc[percentage_missing_mask, 'percentagem_zonaB'] = 0
zonas_com_percentagens.loc[percentage_missing_mask, 'percentagem_zonaA'] = 0

# ver se ficou bem as linhas 231, 75, 40, 186, 20, 104 que tinham nulls
zonas_com_percentagens.iloc[[231, 75, 40, 186, 20, 104]]
# FICOU BEM!!!

,concelho,count_zonaA,count_zonaB,count_zonaC,total_concelho,percentagem_zonaA,percentagem_zonaB,percentagem_zonaC
231,Tavira,0,0,1,1,0.0,0.0,1.0
75,Constância,0,0,1,1,0.0,0.0,1.0
40,Barrancos,0,0,1,1,0.0,0.0,1.0
186,Ponte de Sor,0,0,1,1,0.0,0.0,1.0
20,Alter do Chão,0,0,1,1,0.0,0.0,1.0
104,Guimarães,0,0,1,1,0.0,0.0,1.0


In [14]:
zonas_com_percentagens.rename(columns={"concelho": "Concelho"}, inplace=True)

In [15]:
zonas_com_percentagens.to_excel(here('data/zonas_com_percentagens.xlsx'), index=False)

# Juntar ao conjunto de dados final pelo concelho e exportar

Para corrigir os concelhos no merge estarem com nomes diferentes pus tudo a minúsculas (não é impactante no final)

In [16]:
zonas_com_percentagens["Concelho"] = zonas_com_percentagens["Concelho"].str.lower()

## QuebrasJuntoInc merge

In [17]:
QuebrasJuntoInc = pd.read_excel(here('data/QuebrasJuntoInc.xlsx'))
QuebrasJuntoInc["Concelho"] = QuebrasJuntoInc["Concelho"].str.lower()
QuebrasJuntoInc["Concelho"] = QuebrasJuntoInc["Concelho"].replace('ponte de sôr', 'ponte de sor')
QuebrasJuntoInc

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,EDPD_2018_JAN_T_1,almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,157.9,0.000000,10.7,17.4,2.6,NaN,NaN,NaN,NaN,0.0
1,EDPD_2018_JAN_T_2,sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,260.2,1.000000,10.6,13.8,6.2,249.0,270.0,2.8,8.1,0.0
2,EDPD_2018_JAN_T_3,felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,117.9,0.800000,14.0,14.8,12.9,197.0,180.0,3.9,14.4,4.0
3,EDPD_2018_JAN_T_4,torres vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,156.4,0.923077,12.4,15.1,9.1,288.0,225.0,3.6,13.9,13.2
4,EDPD_2018_JAN_T_5,nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,467.9,1.000000,4.9,7.8,2.2,137.0,118.0,4.1,15.4,15.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,141.7,1.000000,10.3,14.3,6.3,NaN,NaN,NaN,NaN,0.0
7724,EDPD_2017_NOV_T_63,santa maria da feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,134.6,0.000000,16.4,20.6,14.8,NaN,NaN,NaN,NaN,NaN
7725,EDPD_2017_NOV_T_64,santa maria da feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,134.6,0.000000,17.9,22.6,14.6,NaN,NaN,NaN,NaN,8.1
7726,EDPD_2017_NOV_T_65,almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,175.1,1.000000,15.0,20.8,12.8,186.0,186.0,2.5,11.0,NaN


In [18]:
QuebrasJuntoInc_com_zonas = pd.merge(QuebrasJuntoInc, zonas_com_percentagens, on="Concelho", how="left")

In [19]:
QuebrasJuntoInc_com_zonas

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,FF_MED,FF_MAX,PR_QTD,count_zonaA,count_zonaB,count_zonaC,total_concelho,percentagem_zonaA,percentagem_zonaB,percentagem_zonaC
0,EDPD_2018_JAN_T_1,almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,NaN,NaN,0.0,102,63,14,179,0.569832,0.351955,0.078212
1,EDPD_2018_JAN_T_2,sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,2.8,8.1,0.0,0,8,29,37,0.000000,0.216216,0.783784
2,EDPD_2018_JAN_T_3,felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,3.9,14.4,4.0,0,32,13,45,0.000000,0.711111,0.288889
3,EDPD_2018_JAN_T_4,torres vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,3.6,13.9,13.2,0,27,40,67,0.000000,0.402985,0.597015
4,EDPD_2018_JAN_T_5,nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,4.1,15.4,15.9,0,5,6,11,0.000000,0.454545,0.545455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,NaN,NaN,0.0,0,38,18,56,0.000000,0.678571,0.321429
7724,EDPD_2017_NOV_T_63,santa maria da feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,NaN,NaN,NaN,0,32,36,68,0.000000,0.470588,0.529412
7725,EDPD_2017_NOV_T_64,santa maria da feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,NaN,NaN,8.1,0,32,36,68,0.000000,0.470588,0.529412
7726,EDPD_2017_NOV_T_65,almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,2.5,11.0,NaN,0,26,6,32,0.000000,0.812500,0.187500


In [20]:
filtro_quebras = QuebrasJuntoInc_com_zonas["count_zonaA"].isnull()
filtro = QuebrasJuntoInc_com_zonas[filtro_quebras]
print(filtro["Concelho"].nunique())
filtro["Concelho"].unique()

0


array([], dtype=object)

Nenhum concelho sem percentagens e zonas com null
Confirmado no describe abaixo também!

In [27]:
colunas = QuebrasJuntoInc_com_zonas.columns[-7:]
print(f"Número de linhas do dataframe \033[1mQuebrasJuntoInc_com_zonas\033[0m: \033[4m{len(QuebrasJuntoInc_com_zonas)}\033[0m")
QuebrasJuntoInc_com_zonas[colunas].describe()

Número de linhas do dataframe QuebrasJuntoInc_com_zonas: 7728


,count_zonaA,count_zonaB,count_zonaC,total_concelho,percentagem_zonaA,percentagem_zonaB,percentagem_zonaC
count,7728.000000,7728.000000,7728.000000,7728.000000,7728.000000,7728.000000,7728.000000
mean,49.982660,20.197981,17.680642,87.861284,0.173655,0.312379,0.513966
std,155.959946,35.826045,15.899361,156.112884,0.325837,0.305800,0.340966
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,7.000000,15.000000,0.000000,0.000000,0.244444
50%,0.000000,8.000000,13.000000,38.000000,0.000000,0.293103,0.456522
75%,1.000000,26.000000,25.000000,84.000000,0.012987,0.565217,0.819444
max,880.000000,247.000000,106.000000,880.000000,1.000000,1.000000,1.000000


In [32]:
QuebrasJuntoInc_com_zonas.to_excel(here('data/QuebrasJuntoInc_com_zonas.xlsx'), index=False)

## AllDaysEventos merge

In [28]:
AllDaysEventos = pd.read_excel(here('data/AllDaysEventos.xlsx'))
AllDaysEventos["Concelho"] = AllDaysEventos["Concelho"].str.lower()
AllDaysEventos["Concelho"] = AllDaysEventos["Concelho"].replace('ponte de sôr', 'ponte de sor')
AllDaysEventos

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,2014-01-01,albergaria-a-velha,Aveiro,Sem Eventos Excecionais,0,1,1,2014,157.7,25053,...,132.3,1.00,13.9,15.1,12.6,194.0,225.0,4.3,15.4,16.2
1,2014-01-01,anadia,Aveiro,Sem Eventos Excecionais,0,1,1,2014,131.0,28378,...,209.7,1.00,13.9,15.1,12.6,194.0,225.0,4.3,15.4,16.2
2,2014-01-01,arouca,Aveiro,Sem Eventos Excecionais,0,1,1,2014,66.3,21815,...,130.5,1.00,13.9,15.1,12.6,194.0,225.0,4.3,15.4,16.2
3,2014-01-01,aveiro,Aveiro,Sem Eventos Excecionais,0,1,1,2014,400.3,79092,...,124.7,0.00,13.9,15.1,12.6,194.0,225.0,4.3,15.4,16.2
4,2014-01-01,castelo de paiva,Aveiro,Sem Eventos Excecionais,0,1,1,2014,141.5,16275,...,110.9,1.00,13.9,15.1,12.6,194.0,225.0,4.3,15.4,16.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008000,2023-12-31,reguengos de monsaraz,Évora,Sem Eventos Excecionais,0,31,12,2023,21.2,9836,...,208.9,1.00,10.6,13.5,6.8,275.0,339.0,1.6,7.0,1.1
1008001,2023-12-31,vendas novas,Évora,Sem Eventos Excecionais,0,31,12,2023,51.2,11394,...,240.6,1.00,10.6,13.5,6.8,275.0,339.0,1.6,7.0,1.1
1008002,2023-12-31,viana do alentejo,Évora,Sem Eventos Excecionais,0,31,12,2023,13.9,5470,...,197.7,1.00,10.6,13.5,6.8,275.0,339.0,1.6,7.0,1.1
1008003,2023-12-31,vila viçosa,Évora,Sem Eventos Excecionais,0,31,12,2023,37.7,7349,...,238.9,1.00,10.6,13.5,6.8,275.0,339.0,1.6,7.0,1.1


In [29]:
AllDaysEventos_com_zonas = pd.merge(AllDaysEventos, zonas_com_percentagens, on="Concelho", how="left")
AllDaysEventos_com_zonas

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,...,FF_MED,FF_MAX,PR_QTD,count_zonaA,count_zonaB,count_zonaC,total_concelho,percentagem_zonaA,percentagem_zonaB,percentagem_zonaC
0,2014-01-01,albergaria-a-velha,Aveiro,Sem Eventos Excecionais,0,1,1,2014,157.7,25053,...,4.3,15.4,16.2,0,12,7,19,0.000000,0.631579,0.368421
1,2014-01-01,anadia,Aveiro,Sem Eventos Excecionais,0,1,1,2014,131.0,28378,...,4.3,15.4,16.2,0,7,14,21,0.000000,0.333333,0.666667
2,2014-01-01,arouca,Aveiro,Sem Eventos Excecionais,0,1,1,2014,66.3,21815,...,4.3,15.4,16.2,0,0,20,20,0.000000,0.000000,1.000000
3,2014-01-01,aveiro,Aveiro,Sem Eventos Excecionais,0,1,1,2014,400.3,79092,...,4.3,15.4,16.2,112,0,19,131,0.854962,0.000000,0.145038
4,2014-01-01,castelo de paiva,Aveiro,Sem Eventos Excecionais,0,1,1,2014,141.5,16275,...,4.3,15.4,16.2,0,1,10,11,0.000000,0.090909,0.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008000,2023-12-31,reguengos de monsaraz,Évora,Sem Eventos Excecionais,0,31,12,2023,21.2,9836,...,1.6,7.0,1.1,0,9,3,12,0.000000,0.750000,0.250000
1008001,2023-12-31,vendas novas,Évora,Sem Eventos Excecionais,0,31,12,2023,51.2,11394,...,1.6,7.0,1.1,0,10,3,13,0.000000,0.769231,0.230769
1008002,2023-12-31,viana do alentejo,Évora,Sem Eventos Excecionais,0,31,12,2023,13.9,5470,...,1.6,7.0,1.1,0,0,5,5,0.000000,0.000000,1.000000
1008003,2023-12-31,vila viçosa,Évora,Sem Eventos Excecionais,0,31,12,2023,37.7,7349,...,1.6,7.0,1.1,0,10,1,11,0.000000,0.909091,0.090909


In [30]:
filtro_quebras = AllDaysEventos_com_zonas["count_zonaA"].isnull()
filtro = AllDaysEventos_com_zonas[filtro_quebras]
print(filtro["Concelho"].nunique())
filtro["Concelho"].unique()

0


array([], dtype=object)

In [31]:
colunas = AllDaysEventos_com_zonas.columns[-7:]
print(f"Número de linhas do dataframe \033[1mQuebrasJuntoInc_com_zonas\033[0m: \033[4m{len(AllDaysEventos_com_zonas)}\033[0m")
AllDaysEventos_com_zonas[colunas].describe()

Número de linhas do dataframe QuebrasJuntoInc_com_zonas: 1008005


,count_zonaA,count_zonaB,count_zonaC,total_concelho,percentagem_zonaA,percentagem_zonaB,percentagem_zonaC
count,1.008005e+06,1.008005e+06,1.008005e+06,1.008005e+06,1.008005e+06,1.008005e+06,1.008005e+06
mean,1.269078e+01,1.247456e+01,1.259826e+01,3.776361e+01,6.871506e-02,2.694499e-01,6.618350e-01
std,6.622566e+01,2.630837e+01,1.200115e+01,7.441623e+01,2.161838e-01,3.148379e-01,3.463548e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,5.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,3.684211e-01
50%,0.000000e+00,2.000000e+00,9.000000e+00,1.500000e+01,0.000000e+00,6.382979e-02,7.073171e-01
75%,0.000000e+00,1.400000e+01,1.500000e+01,4.100000e+01,0.000000e+00,5.522388e-01,1.000000e+00
max,8.800000e+02,2.470000e+02,1.060000e+02,8.800000e+02,1.000000e+00,1.000000e+00,1.000000e+00


In [34]:
AllDaysEventos_com_zonas.to_feather(here('data/AllDaysEventos_com_zonas.feather'))